In [1]:
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(DescTools))
suppressPackageStartupMessages(library(scales))
suppressPackageStartupMessages(library(parallel))

home_dir  = '/nfs/lab/projects/selex/pipeline'
#well      = basename(getwd())
prot      = read.table(paste(home_dir, "TF_list.txt", sep="/"), header=T, stringsAsFactors = F)
well_list = prot$well
setwd(home_dir)

In [2]:
#### I copied the count files in a new directory
# for( well in well_list){
# dir.create(wwll)
# dir.create(paste0(well, '/snvs'))
# file1 = paste('/nfs/lab/projects/selex/selex_t2d_t1d/results/obs_pbs_snvs/', well,  "/Sum_of_alleles_over_cycles*", sep="")
# system(paste('cp', file1, paste0(well, '/snvs')))
# dir.create(paste0(well, '/indels'))
# file2 = paste('/nfs/lab/projects/selex/selex_t2d_t1d/results/obs_pbs_indels/', well,  "/Sum_of_alleles_over_cycles*", sep="")
# system(paste('cp', file2, paste0(well, '/indels')))       
# }       

In [2]:
###### FUNCTIONS ######
######################
### function to calculate the area used for PBS and OBS

calc_area = function(xx, yy){
  N   = length(xx)
  dx  = xx[2:N] - xx[1:(N-1)]
  my  = 0.5*(yy[2:N] + yy[1:(N-1)])
  A   = sum(dx * my)
return(A)
    }

##### function to caluclate PBS 
calc_pbs = function(total_counts, ref_counts, alt_counts, pseudo =0.1){

tab_ref = data.frame( "rest" = total_counts - ref_counts,  "reads" = ref_counts ) + pseudo
tab_alt = data.frame( "rest" = total_counts - alt_counts,  "reads" = alt_counts ) + pseudo
    
logOR_ref = sapply(1:5, function(p) log10(tab_ref[1,1]) + log10(tab_ref[p,2]) - log10(tab_ref[1,2]) -log10(tab_ref[p,1]))
logOR_alt = sapply(1:5, function(p) log10(tab_alt[1,1]) + log10(tab_alt[p,2]) - log10(tab_alt[1,2]) -log10(tab_alt[p,1]))  
    
pbs = calc_area(0:4, logOR_ref) - calc_area(0:4, logOR_alt)  
return(pbs)    
}

#### make permutations by changing labels of the snps within the same cycles

permuted_vals = function(MER){
    
permuted_snps =  unlist(lapply(split(MER, MER$cycle), function(x)  sample(x$snp_var )))
o             =  sample(permuted_snps,1) 
test          =  subset(MER, permuted_snps==o)
dd            = c( calc_pbs(test$read_count, test$ref_count, test$alt_count),
                   calc_pbs(test$total_reads, test$ref_count, test$alt_count))
return(dd)                             
}

In [3]:
##### Test using one experiment (loop or parallelize for all wells, two replicates and 2 variant types)
well    = "8_D3"  ### well_list
rep     = "rep1"  ### or "rep2" or none for merged 
vtype   = 'snvs'  ### or indels
file    = paste(well, "/" ,vtype,   "/Sum_of_alleles_over_cycles_", rep, ".txt", sep="")



In [18]:
if(file.exists(file)) {
MER = read.table(file, header=T,  sep="\t")

if((nrow(MER)>100 & vtype == "snvs") | (nrow(MER) >=10   & vtype == "indels" ))   { ### minumum number of oligos x well

ag  = aggregate(cycle~snp_var, MER, paste, collapse="")
MER = subset(MER,snp_var %in% ag$snp_var[ag$cycle=="01234"]) ### read counts in all cycles

PBS = OBS = c()
#### loop thorugh each variant
for( o in unique(MER$snp_var)) {
        test = subset(MER, snp_var==o)

    if (test$ref_count[1]/test$read_count[1] >= 0.05 & test$alt_count[1]/test$read_count[1] >= 0.05 ) { 
    ## ref and alt allele should be represented by at least 5% of input reads

        ### calculate PBS #########
        PBS[o]   = calc_pbs(test$read_count, test$ref_count, test$alt_count)

        ### calculate deltaOBS ## the only difference is that is calculated on the total reads 
        OBS[o]   = calc_pbs(test$total_reads, test$ref_count, test$alt_count)    
        } 
         }                  
d = data.frame(OBS, PBS)

    
np=250000  #### number of permutations

perm = mclapply(1:np, function(x) permuted_vals(MER), mc.cores=18)
perm = matrix(unlist(perm), ncol=2, byrow=T)
colnames(perm) = c("rPBS", "rOBS")

write.table(perm, paste(well,"/" ,vtype,"/permutations_",rep, sep=""), quote=F, col.names=T, row.names=F)

### P-values
d$pv_OBS =  (sapply( abs(d$OBS), function(x) sum( abs(perm[,2])  >=x ))+1)/(np+1)
d$pv_PBS =  (sapply( abs(d$PBS), function(x) sum( abs(perm[,1])  >=x ))+1)/(np+1)

### Q-values
d$qv_OBS = p.adjust(d$pv_OBS, "BH")
d$qv_PBS = p.adjust(d$pv_PBS, "BH")

write.table(d, paste(well,"/Enrichment_scores_", vtype ,'.',rep, sep=""), quote=F)
                    }
                    }
                    
                    
                    

In [29]:
head(test)

,snp,A,T,G,C,read_count,total_reads,cycle,ref,alt,t2d,t1d,snp_var,ref_count,alt_count,permuted_snps
,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<lgl>,<lgl>,<fct>,<int>,<int>,<fct>
1,T1D_1_101634618_G,5,9,4,2,20,1567611,0,G,A,TRUE,TRUE,T1D_1_101634618_G_A,4,5,T1D_6_32596647_T_C
13494,T2D_11_92812922_A,9,10,10,8,37,638181,1,A,G,TRUE,FALSE,T2D_11_92812922_A_G,9,10,T1D_6_32596647_T_C
23792,T2D_4_96748556_T,9,18,8,4,39,575048,2,T,A,TRUE,FALSE,T2D_4_96748556_T_A,18,9,T1D_6_32596647_T_C
26421,T1D_16_29198117_G,8,8,6,6,28,551284,3,G,A,FALSE,TRUE,T1D_16_29198117_G_A,6,8,T1D_6_32596647_T_C
37496,T1D_7_27017870_G,8,15,6,9,38,655179,4,G,A,FALSE,TRUE,T1D_7_27017870_G_A,6,8,T1D_6_32596647_T_C


#### Count the sum of the reads in cycle 1:4 to use as weight in the metaanalysis
I forgot to add it so I'll make a new file

In [ ]:
for (well   in    well_list)     {
for (rep    in    c("rep1" , "rep2" ))   {
for (vtype  in    c('snvs' , 'indels'))      {

file    = paste(well, "/" ,vtype,   "/Sum_of_alleles_over_cycles_", rep, ".txt", sep="")

if(file.exists(file)) {
MER = read.table(file, header=T,  sep="\t")

if((nrow(MER)>100 & vtype == "snvs") | (nrow(MER) >=10   & vtype == "indels" ))   { ### minumum number of oligos x well

ag  = aggregate(cycle~snp_var, MER, paste, collapse="")
MER = subset(MER,snp_var %in% ag$snp_var[ag$cycle=="01234"]) ### read counts in all cycles

sp = split(MER, MER$snp_var)
readcounts = sapply( sp, function(x) sum(x[2:5,c('ref_count','alt_count')]))
     outfile = paste(well, "/" ,vtype,   "/read_counts_", rep, ".txt", sep="") 
                    write.table (readcounts, outfile, col.names=F, sep="\t", quote=F)             
                    }
            }
                  }
                    }
                      }